# Imports

In [3]:
# reinstall dependencies for the current notebook environment because we are using a python 3.11.0 kernel
%pip install pandas ViennaRNA

import pandas as pd
import RNA


[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 11.3/11.3 MB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 12.6/12.6 MB 5.1 MB/s eta 0:00:00
     -------------------------------------- 509.2/509.2 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 348.5/348.5 kB 5.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ImportError: Failed to load ViennaRNA RNAlib Python wrapper

# Load Raw Data

In [ ]:
file_name = 'train1'
df = pd.read_csv(f'../data/raw/{file_name}.csv')
print(df.head())

# Compute MFE with nupack

In [ ]:
model = nupack.Model(material='dna')

df['nupack_mfe_energy'] = df['promoter'].apply(lambda seq: nupack.mfe(sequences=[seq], model=model)[0].energy)
df['nupack_mfe_structure'] = df['promoter'].apply(lambda seq: nupack.mfe(sequences=[seq], model=model)[0].structure)

# Print some results
print(df[['promoter', 'nupack_mfe_energy', 'nupack_mfe_structure']].head())

# Compute MFE with ViennaRNA

In [ ]:
df['viennaRNA_mfe_energy'] = df['promoter'].apply(lambda seq: RNA.fold(seq)[1])

# Print some results
print(df[['promoter', 'viennaRNA_mfe_energy']].head())

# Compute MFE for windows using both libraries

In [ ]:
promoter_length = len(df.loc[0, 'promoter'])
win_size = 100
window_coordinates = [[0,500],[500,800],[800,1000]].concat([[i*win_size, (i+1)*win_size] for i in range(0, int(promoter_length/win_size))])

for (start,end) in window_coordinates:
    window = df['promoter'].str[start:end]

    # Compute the NUPACK MFE for the window
    df[f'nupack_mfe_energy_window_{start}_{end}'] = df['promoter'].apply(lambda seq: nupack.mfe(sequences=[seq], model=model)[0].energy)

    # Compute the MFE using viennaRNA
    df[f'viennaRNA_mfe_energy_window_{start}_{end}'] = df['promoter'].apply(lambda seq: RNA.fold(seq)[1])

print(df.head())